# Notebook #1: Extracción: extracción web de los datos

In [1]:
# Importamos las librerías que necesitamos

# Librerías de extracción de datos
# -----------------------------------------------------------------------

%load_ext autoreload
%autoreload 2

# Visualizaciones
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
import plotly_express as px

import sys
sys.path.append('../..')
from src import funciones as f
from src import soporte_prep as sp

# Importaciones:
# Beautifulsoup
from bs4 import BeautifulSoup

# Requests
import requests

import pandas as pd
import numpy as np

from time import sleep

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 

import warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_columns', None)

In [2]:
driver = webdriver.Chrome()
url_accidentes = "https://www.one.gob.do/datos-y-estadisticas/"
driver.get(url_accidentes)
driver.maximize_window()

sleep(1)  # Esperar un segundo entre cada scroll

try: 
    driver.find_element("xpath", '/html/body/div[7]/section/div/ul/li[2]/a').click()
except:
    print('No encuentro el botón')

sleep(2)

for i in range(2):
    try:
        driver.execute_script("window.scrollBy(0, 500);")  # Desplazarse 1000 píxeles hacia abajo
    except:
        print('No he podido descender')

try: 
    driver.find_element("xpath", '/html/body/div[7]/section/div/div[3]/div[2]/div/div[2]/div[2]/div/div/div[2]/header/div[2]').click()
except:
    print('No encuentro el botón para ampliar')
sleep(2)
try: 
    driver.find_element("xpath", '/html/body/div[7]/section/div/div[3]/div[2]/div/div[2]/div[2]/div/div/div[2]/div/header/div[1]').click()
except:
    print('No encuentro el botón para abrir pestañas')
sleep(2)
try:
    driver.execute_script("window.scrollBy(0, 700);")
except:
    print('No he podido descender')

try: 
    driver.find_element("xpath", '/html/body/div[7]/section/div/div[3]/div[2]/div/div[2]/div[2]/div/div/div[2]/div/div/div/div/div[2]/div/table/tbody/tr[6]/td/a').click()
except:
    print('No encuentro el botón para descargar') #descargo el libro de códigos de accidentes de tráfico, lasdefunciones

try:
    driver.execute_script("window.scrollBy(0, 500);")
except:
    print('No he podido descender')

try: 
    driver.find_element("xpath", '/html/body/div[7]/section/div/div[3]/div[2]/div/div[2]/div[2]/div/div/div[2]/div/div/div/div/div[2]/div/table/tbody/tr[17]/td/a').click()
except:
    print('No encuentro el botón para descargar') #descargo las defunciones por accidentes de tráfico

## Tablas secundarias

In [3]:
driver = webdriver.Chrome()
url_defunciones = "file:///C:/Users/DELL/Downloads/Libro%20de%20c%C3%B3digos%20de%20defunciones%20en%20accidentes%20de%20tr%C3%A1nsito%202007_2023.htm"
driver.get(url_defunciones)
driver.maximize_window()

sleep(1)  # Esperar un segundo entre cada scroll

In [4]:
sopa_defunciones = BeautifulSoup(driver.page_source)
lista_tablas = sopa_defunciones.findAll('div', {'class': 'itemBody'})

dataframes = []

for i, tabla in enumerate(lista_tablas):
    rows = tabla.find_all('tr')

    data = []
    for row in rows:
        cells = row.find_all('td')
        data.append([cell.get_text(strip=True) for cell in cells if cell.get_text(strip=True)])

    df = pd.DataFrame(data)

    dataframes.append(df)

print(f"Se generaron {len(dataframes)} DataFrames.")

Se generaron 21 DataFrames.


In [5]:
df_region = dataframes[1]
df_provincia = dataframes[2]
df_diasem = dataframes[3]
df_mes = dataframes[4]
df_rhora = dataframes[6]
df_tipovia = dataframes[7]
df_genero = dataframes[8]
df_gedad = dataframes[9]
df_victima = dataframes[10]
df_accidente = dataframes[11]
df_mediot = dataframes[12]

In [6]:
dataframes_list = [
    df_region, df_provincia, df_diasem, df_mes, df_rhora,
    df_tipovia, df_genero, df_gedad, df_victima, df_accidente,
    df_mediot
]

for i, df in enumerate(dataframes_list):
    # me quedo con las filas a partir de la fila 3 porque así me quedo solo con el contenido
    df = df.iloc[3:]
    
    # coloco el primer valor en su lugar, ya que viene en la columna equivocada
    df.iloc[0, 1] = df.iloc[0, 2]
    df.iloc[0,0] = 1
    
    # me quedo solo la primera columna
    df = df[[0,1]]
    df = df[:-1]
    df.columns = ['Valor', 'Significado']
    df['Valor'] = df['Valor'].astype('int64')

    # reinicio el índice
    df.reset_index(drop=True, inplace=True)
    
    # guardo el df transformado de nuevo en la lista
    dataframes_list[i] = df

# Asignar los DataFrames de vuelta a sus variables
df_region, df_provincia, df_diasem, df_mes, df_rhora, \
df_tipovia, df_genero, df_gedad, df_victima, df_accidente, \
df_mediot = dataframes_list

In [8]:
dataframes = [
    (df_diasem, 'df_diasem.csv'),
    (df_mes, 'df_mes.csv'),
    (df_rhora, 'df_rhora.csv'),
    (df_region, 'df_region.csv'),
    (df_provincia, 'df_provincia.csv'),
    (df_tipovia, 'df_tipovia.csv'),
    (df_accidente, 'df_accidente.csv'),
    (df_mediot, 'df_mediot.csv'),
    (df_genero, 'df_genero.csv'),
    (df_gedad, 'df_gedad.csv'),
    (df_victima, 'df_victima.csv')
]

for df, filename in dataframes:
    df.to_csv(f'../../Datos/datos_defunciones/{filename}')
